# Sets

In [ ]:
{1,2,3}

In [ ]:
{1,3,2,3,3}

In [ ]:
my_set = {"a", 2, (1,2)}

In [ ]:
my_set.add("a")

In [ ]:
my_set

# Static methods and classmethods

In [ ]:
from math import sin, cos, pi, sqrt
class Point(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    @classmethod
    def from_polar(cls, r, phi):
        return cls(r * cos(phi),
                   r * sin(phi))
    @staticmethod
    def r(p):
        return sqrt(p.x**2 + p.y**2)
Point.from_polar(1, pi)

In [ ]:
Point.r(Point.from_polar(1.5, pi/2))

## More special methods

In [ ]:
class PointWithAccess(Point):
    def __init__(self, x, y):
        super(PointWithAccess, self).__init__(x, y)
    def __getitem__(self, i):
        if i == 0:
            return self.x
        elif i == 1:
            return self.y
        else:
            raise KeyError
    def __iter__(self):
        yield self.x
        yield self.y

In [ ]:
p = PointWithAccess(1, 2)
print p

In [ ]:
p[0], p[1]

In [ ]:
[i for i in p]

# Generators

In [ ]:
def get_messages():
    messages = {'One message.', 'Another message.'}
    while messages:
        yield messages.pop()

In [ ]:
get_messages()

In [ ]:
[i for i in get_messages()]

In [ ]:
i = get_messages()

In [ ]:
next(i)

In [ ]:
i.next()

In [ ]:
next(i)

# Error handling

In [ ]:
class MeanValue(object):
    n = 0
    total = 0
    def put(self, x):
        n += 1
        total += x
    def get(self):
        if self.n == 0:
            raise ValueError('No values stored!')
        else:
            return self.total / self.n

In [ ]:
m = MeanValue()

In [ ]:
m.get()

In [ ]:
mean = None
try:
    mean = m.get()
except ValueError as e:
    print e

# Scipy / Numpy

**Numpy** provides:

- An `array` class.
- Random sampling.
- Legacy stuff.

**Scipy** provides:

- Interpolation.
- Optimizaiton.
- Numerical integration.
- Fourier transformaions.
- Signal processing.
- Linear algebra.

In [ ]:
import numpy as np
import scipy as sp

In [ ]:
np.array(range(10), dtype=np.int16)

In [ ]:
np.array(range(10), dtype=np.float32)

In [ ]:
np.arange(0, 10, .5)

In [ ]:
np.array(range(10)) ** np.array(range(10))

## Why arrays?

In [ ]:
my_array = np.random.standard_normal(1000)
%timeit sum(my_array**2)

In [ ]:
my_list = list(my_array)
%timeit sum([i**2 for i in my_array])

In [ ]:
y = np.random.standard_normal(10)

In [ ]:
x = np.arange(10)

In [ ]:
x,y

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pylab as plt

# Interpolation

## Splines

A spline $S$ is a piecewise polynomial function on an interval [a, b], i.e. there exists an $n$ and $x_i \in [a,b]$ with $a = x_0 < x_2 < \ldots < x_n = b$, such that

$\left. S\right|_{[x_{i-1}, x_i)} = P_i, \quad i = 1,\ldots, n$

with $P_i$ polynomial and $P_i(x_i) = P_{i+1}(x_i)$.

In [ ]:
from scipy.interpolate import interp1d

In [ ]:
S = interp1d(x,y,kind='cubic')

In [ ]:
S(x)

In [ ]:
plt.plot(x, y, 's', np.arange(0,9.1,.1), S(np.arange(0,9.1,.1)), "-")

http://www.aavso.org/data-download

# Intergration

We now numerically approximate

$$\int_2^5 S(x) {\rm d}x\,.$$


In [ ]:
from scipy.integrate import quad

In [ ]:
quad(S, 2, 5)

# Differential equations

In [ ]:
from scipy.integrate import odeint

$$\theta''(t) + b\theta'(t) + c \sin(\theta(t)) = 0$$
$$\Leftrightarrow \theta'(t) = \omega(t), \quad \omega'(t) = -b \omega(t) - c \sin(\theta(t)$$

In [ ]:
def pend(y, t, b, c):
    theta, omega = y
    dydt = [omega, -b*omega - c*np.sin(theta)]
    return dydt

In [ ]:
b,c = 0.25, 5

In [ ]:
y0 = [np.pi - .01, 0.0]

In [ ]:
t = np.arange(0, 10, 0.1)

In [ ]:
solution = odeint(pend, y0, t, args=(b, c))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(t, solution[:, 0], label=r'$\theta$')
plt.plot(t, solution[:, 1], label=r'$\omega$')
plt.legend()
plt.xlabel('t')

# Optimization

## Least-square fitting

Let $f_\theta: \mathbb{R} \rightarrow \mathbb{R}$, $\theta \in \mathbb{R}^k$ be a family of functions, e.g.

$$f_\theta(x) = \theta_0 + \theta_1 x + \theta_2 x^2\,.$$

Let $x_i, y_i, i = 1,\ldots,n$ be observations of some kind, e.g. temperatures $y$ measuread at times $x$.

Our **objective** is to find

$$\theta^* = \underset{\theta}{\rm argmin} \sum_i \left(y_i - f_\theta(x_i)\right)^2\,.$$

In [ ]:
import scipy.optimize

https://archive.ics.uci.edu/ml/machine-learning-databases/housing/

In [ ]:
boston = np.recfromtxt('../data/housing.tsv', names=True)

In [ ]:
pd.read_csv('../data/housing.tsv', sep='\s+').cov()

In [ ]:
plt.scatter(boston.TAX, boston.MEDV)
plt.xlabel("TAX")
plt.ylabel("MEDV")

In [ ]:
f = lambda x, a, b: a*x + b

In [ ]:
(a, b), _ = scipy.optimize.curve_fit(f, boston.TAX, boston.MEDV)

In [ ]:
plt.plot(boston.TAX, boston.MEDV, 'o')
x = np.linspace(boston.TAX.min(), boston.TAX.max(), 100)
plt.plot(x, f(x, a, b))

In [ ]:
g = lambda x, a, b, c: a*x**2 + b*x + c

In [ ]:
(i,j,k), _ = scipy.optimize.curve_fit(g, boston.TAX, boston.MEDV)

In [ ]:
plt.plot(boston.TAX, boston.MEDV, 'o')
x = np.linspace(boston.TAX.min(), boston.TAX.max(), 100)
plt.plot(x, f(x, a, b))
plt.plot(x, g(x, i, j, k))

In [ ]:
h = lambda theta, x: theta[0] + x*theta[1] + x*x*theta[2]
errfn = lambda theta: sum((boston.MEDV - h(theta, boston.TAX))**2)
params = scipy.optimize.minimize(errfn, [1,1,1])

In [ ]:
params

In [ ]:
plt.plot(boston.TAX, boston.MEDV, 'o')
plt.plot(x, h(params.x, x))
plt.xlabel("TAX")
plt.ylabel("MEDV")

# Fourier transform

$$y_k = \sum_j e^{-2 \pi i \frac {kj} N} x_j$$

Heart series data: http://ecg.mit.edu/time-series/


In [ ]:
heart_1 = np.genfromtxt('data/hr_series_1.txt')
heart_2 = np.genfromtxt('data/hr_series_2.txt')

In [ ]:
from scipy.fftpack import fft

In [ ]:
y_1, y_2 = [abs(fft(i)[1:200])**2 for i in (heart_1, heart_2)]

In [ ]:
plt.plot(y_1)
plt.plot(y_2)
plt.xlabel("inverse frequency")
plt.ylabel("power")